In [12]:
import sys, os
import timeit
sys.path.append(os.path.dirname(os.getcwd()))
import numpy as np
import pythia
import pythia.regions as regions
import pythia.interaction as interaction
import matplotlib.pyplot as plt
# from nodegam.sklearn import NodeGAMClassifier, NodeGAMRegressor
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingRegressor


# hack to reload modules
import importlib
pythia = importlib.reload(pythia)


class RepidSimpleDist:
    """
    x1 ~ U(-1, 1)
    x2 ~ U(-1, 1)
    x3 ~ Bernoulli(0.5)
    """

    def __init__(self):
        self.D = 2
        self.axis_limits = np.array([[-1, 1], [-1, 1], [0, 1]]).T

    def generate(self, N):
        x1 = np.concatenate((np.array([-1]),
                             np.random.uniform(-1, 1., size=int(N-2)),
                             np.array([1])))
        x2 = np.concatenate((np.array([-1]),
                             np.random.uniform(-1, 1., size=int(N-2)),
                             np.array([1])))
        x3 = np.random.choice([0, 1], int(N), p=[0.5, 0.5])
        x4 = np.random.choice([0, 1], int(N), p=[0.5, 0.5])

        x = np.stack((x1, x2, x3, x4), axis=-1)
        return x


class RepidSimpleModel:
    def __init__(self, a1=0.2, a2=-8, a3=8, a4=8, a5=0):
        self.a1 = a1
        self.a2 = a2
        self.a3 = a3
        self.a4 = a4
        self.a5 = a5

    def predict(self, x):
        y = np.zeros_like(x[:,0])
        
        cond = np.logical_and(np.logical_and(x[:, 0] > 0, x[:, 2] == 0), x[:, 3] == 0)
        y[cond] += self.a5*x[cond, 1]

        cond = np.logical_and(np.logical_and(x[:, 0] > 0, x[:, 2] == 0), x[:, 3] > 0)
        y[cond] -= self.a5*x[cond, 1]

        eps = np.random.normal(loc=0, scale=0.1, size=y.shape[0])
        y += eps
        return y

    def jacobian(self, x):
        y = np.stack([self.a1*np.ones(x.shape[0]), self.a2*np.ones(x.shape[0]), np.ones(x.shape[0]), np.ones(x.shape[0])], axis=-1) * 0

        cond = np.logical_and(np.logical_and(x[:, 0] > 0, x[:, 2] == 0), x[:, 3] == 0)
        y[cond, 1] += self.a5
        
        cond = np.logical_and(np.logical_and(x[:, 0] > 0, x[:, 2] == 0), x[:, 3] > 0)
        y[cond, 1] -= self.a5*x[cond, 1]
        return y


def plot_effect_ebm(ebm_model, ii):
    explanation = ebm_model.explain_global()
    plt.figure(figsize=(10, 6))
    xx = explanation.data(ii)["names"][:-1]
    yy = explanation.data(ii)["scores"]
    plt.xlim(-1, 1)
    plt.ylim(-4, 4)
    plt.plot(xx, yy)
    plt.show()


np.random.seed(21)
feature_names = ["x1", "x2", "x3"]
dist = RepidSimpleDist()
model = RepidSimpleModel()

# generate data
X = dist.generate(N=1000)
Y = model.predict(X)

In [16]:
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())
from interpret import show


gam_interactions = ExplainableBoostingRegressor()
gam_interactions.fit(X, Y)
print(gam_interactions.score(X, Y))
explanation = gam_interactions.explain_global()
show(explanation)


0.007493801608230988


In [14]:
X.shape

(1000, 4)